# Построение неперсонализованной рекомендательной системы

Цель работы: создать модель рекомендательной системы, основываясь на популярности статей.

Результаты:

1. Создана модель рекомендаций на основе популярности. Пользователю рекомендуется список из самых популярных статей. 

2. При этом в рекомендованных статьях есть только те статьи, с которыми пользователь ещё не взаимодействовал.

3. Произведена оценка качества рекомендаций с помощью метрики Precision@10 (точность с отсечением).

In [21]:
#Импорт нужных библиотек

import pandas as pd
import numpy as np
import math

## Знакомство с данными о статьях


In [22]:
shared_articles = pd.read_csv('shared_articles.csv')
shared_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


## Первичная фильтрация данных о статьях

Отфильтруем данные так, чтобы остались только объекты с типом события CONTENT SHARED. 

Посчитаем, сколько таких объектов в получившейся таблице.

In [23]:
sa_shared = shared_articles[shared_articles['eventType'] == 'CONTENT SHARED']
sa_shared.shape[0]

3047

## Знакомство с данными о взаимодействии пользователя со статьями

Откроем второй файл:

In [24]:
users_interactions = pd.read_csv('users_interactions.csv')
users_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:

- VIEW — просмотр,
- LIKE — лайк,
- COMMENT CREATED — комментарий,
- FOLLOW — подписка,
- BOOKMARK — добавление в закладки.

В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. 

Составим его из реакций пользователей, придав им разные веса (см. код ниже). 

Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

In [25]:
#Зададим веса для каждого типа взаимодействия пользователя со статьёй

event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

## Создание числового признака для оценки взаимодействия пользователя со статьёй

Создадим признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Оценим среднее значение для полученного признака.

In [26]:
users_interactions['eventWeight'] = users_interactions['eventType'].apply(lambda x: event_type[x])

users_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventWeight
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0


In [27]:
users_interactions['eventWeight'].mean()

1.2362885828078327

## Ограничение числа пользователей для увеличения информативности

Чтобы получить действительно значимую информацию, на которую можно будет опираться, оставим только тех пользователей, которые взаимодействовали минимум с пятью статьями. 

Оценим число таких пользователей.

In [28]:
users_df = (
    users_interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
 
users_5_df = users_df[users_df > 4].reset_index()[['personId']]
print(len(users_5_df))

1140


## Фильтрация данных о взаимодействиях

Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Оценим количество таких взаимодействий в общем.


In [29]:
#В полученном датафрейме filtered представлены только пользователи, которые взаимодействовали со статьями не менее 5 раз.
filtered = users_interactions[users_interactions['personId'].isin(users_5_df['personId'])]
filtered.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventWeight
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0


In [30]:
filtered.shape[0]

69868

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. 

Для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат. 

Выполним такое логарифмирование с помощью следующей функции:

In [31]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

## Преобразование данных по весам для каждой статьи + фиксация времени последнего взаимодействия

Применим упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраним для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Чтобы выбрать последнее по времени взаимодействие, используем метод max().

In [32]:
#Группировка по пользователям и статьям + логарифмирование суммы признака взаимодействия
gr = filtered.groupby(['personId','contentId'])['eventWeight'].sum().apply(smooth_user_preference).reset_index().set_index(['personId', 'contentId'])
gr

eventWeight
personId             contentId                        
-9223121837663643404 -8949113594875411859     1.000000
                     -8377626164558006982     1.000000
                     -8208801367848627943     1.000000
                     -8187220755213888616     1.000000
                     -7423191370472335463     3.169925
...                                                ...
 9210530975708218054  8477804012624580461     3.247928
                      8526042588044002101     1.000000
                      8856169137131817223     1.000000
                      8869347744613364434     1.000000
                      9209886322932807692     1.000000

[39106 rows x 1 columns]

In [33]:
#Выделение и добавление признака времени (момент последнего взаимодействия - максимальное время)
#Важно: здесь мы имеем место с добавлением нового столбца в датафрейм с мультииндексом. 
#Новый столбец тоже должен иметь соответствующий мультииндекс.

gr['last_timestamp'] = filtered.groupby(['personId','contentId'])['timestamp'].max()
gr

eventWeight  last_timestamp
personId             contentId                                        
-9223121837663643404 -8949113594875411859     1.000000      1462452127
                     -8377626164558006982     1.000000      1473938707
                     -8208801367848627943     1.000000      1469706702
                     -8187220755213888616     1.000000      1467823897
                     -7423191370472335463     3.169925      1479376578
...                                                ...             ...
 9210530975708218054  8477804012624580461     3.247928      1486577729
                      8526042588044002101     1.000000      1482887760
                      8856169137131817223     1.000000      1476790903
                      8869347744613364434     1.000000      1481294993
                      9209886322932807692     1.000000      1477409052

[39106 rows x 2 columns]

## Промежуточный итог 1

Мы получили информацию о взаимодействии каждого пользователя с каждой отдельно взятой статьёй. 

При этом мы можем оценить силу взаимодействия (вес) и точное время последнего взаимодействия с данной статьёй.

В этом проекте мы не будем использовать данные о времени, хотя для некоторых задач это было бы весьма полезно.

## Разбиение данных на тренировочные и тестовые

Для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.

Разделим данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Значение отсечки включим в тестовую выборку. 

Оценим количество объектов, попавших в тестовую и обучающую выборку.

In [34]:
#Разбиение данных на тестовые и тренировочные в соотношении 1 к 3

test_df = gr[gr['last_timestamp'] >= 1475519545].copy()
train_df = gr[gr['last_timestamp'] < 1475519545].copy()
print(test_df.shape[0])
print(train_df.shape[0])

9781
29325


## Преобразование данных

Для удобства дальнейшего измерения качества рекомендаций преобразуем данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместим пустые списки.

In [35]:
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]

final_df.head()

,true_train,true_test
personId,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[]
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084]
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]"
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -..."


## Построение popular-based модели

### Находим самые популярные статьи

Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитаем популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). 

Определим ID самой популярной статьи.

In [36]:
#Считаем популярность каждой статьи
most_pop = train_df.groupby('contentId')['eventWeight'].sum().reset_index()
most_pop

,contentId,eventWeight
0,-9222795471790223670,15.089906
1,-9216926795620865886,13.754888
2,-9192549002213406534,52.339850
3,-9190737901804729417,3.321928
4,-9189659052158407108,29.509775
...,...,...
2361,9209629151177723638,5.000000
2362,9215261273565326920,22.674053
2363,9217155070834564627,11.044394
2364,9220445660318725468,36.299208


In [37]:
#Сортируем статьи по популярности и находим самые популярные

most_pop.sort_values(by='eventWeight', ascending=False).iloc[:10]

,contentId,eventWeight
327,-6783772548752091658,231.177195
1158,-133139342397538859,228.024567
151,-8208801367848627943,189.937683
2235,8224860111193157980,186.044680
2144,7507067965574797372,179.094002
869,-2358756719610361882,175.771101
317,-6843047699859121724,175.108147
1006,-1297580205670251233,160.671086
2289,8657408509986329668,157.973460
1609,3367026768872537336,149.383615


In [38]:
#Получим массив идентификаторов статей, отсортированный по убыванию, для дальнейших действий

popular = (
    train_df
    .groupby('contentId')
    .eventWeight.sum().reset_index()
    .sort_values('eventWeight', ascending=False)
    .contentId.values
)
popular[0]

-6783772548752091658

### Создание списка рекомендованных статей для каждого пользователя

Будем рекомендовать десять самых популярных статей из тех, которые он ещё не читал.

То есть для каждого пользователя будет создан свой уникальный список из 10 статей. Списки рекомендаций будут во многом похожи, но всё же будут отличаться одной или несколькими статьями.


В предыдущем задании мы посчитали популярность каждой статьи и записали отсортированные по убыванию значения популярности в массив popular. В коде ниже мы генерируем рекомендации для каждого пользователя из обучающей выборки и записываем их в столбец popular датафрейма. Для каждого пользователя мы:

- Берём те статьи из массива popular, которые этот пользователь ещё не читал (popular[~np.in1d(popular, x)]).
- Из них выбираем k самых популярных.


In [39]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)

final_df.head()

,true_train,true_test,popular
personId,,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -...","[-6783772548752091658, -133139342397538859, 82..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[],"[-6783772548752091658, -133139342397538859, -8..."
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084],"[-6783772548752091658, -133139342397538859, -8..."
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]","[-6783772548752091658, -133139342397538859, -8..."
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -...","[-6783772548752091658, -133139342397538859, -8..."


## Оценка качества рекомендаций

Оценим качество с помощью precision@10 для каждого пользователя (доля корректных рекомендаций). После этого усредним результат по всем пользователям.

Precision - это отношение количества релевантных рекомендаций к общему количеству рекомендованных элементов (в нашем случае не более 10).

Для вычисления precision@10 воспользуемся следующей функцией:


In [40]:
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

calc_precision('popular')

0.006454207722621083

## Результаты работы

1. Построена модель рекомендации статей для разных пользователей на основе популярности статей у всех пользователей.

2. Каждому пользователью рекомендуется 10 статей из самых популярных, за исключением тех, с которыми он уже взаимодействовал. 

3. Выведена оценка точности прогноза самых популярных статей на основании тестовых данных. Оценка получилась низкой - около 0.006 по выбранной методике оценки. 

4. Низкая оценка связана, прежде всего, с отсутствием персонализации рекомендаций. 

5. Для повышения качества рекомендаций стоит применить другие методы с персонализацией.